Fetching

In [3]:
import pandas as pd
import os
from PIL import Image
import numpy as np
import cv2
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


labels_df = pd.read_csv('G:\\Graduation Project\\trainLabels.csv')
labels_df['inverted'] = False
print(labels_df.head())

image_dir = 'G:\\Graduation Project\\Images\\train\\' 

def load_image(image_name):
    file_extension = '.jpeg'
    filename = f"{image_name}{file_extension}"
    file_path = os.path.join(image_dir, filename)
    return Image.open(file_path)

for i in range (1):
    sample_image = load_image(labels_df.iloc[i]['image'])
    sample_image.show()

      image  level  inverted
0   10_left      0     False
1  10_right      0     False
2   13_left      0     False
3  13_right      0     False
4   15_left      1     False


Inversion checking

In [108]:
def is_inverted(image):
    try:
        # Ensure the image is fully loaded
        image.load()
    except Exception as e:
        print(f"Error loading image data: {e}")
        raise e  # Re-raise the exception

    try:
        # Ensure image is in RGB mode
        if image.mode != 'RGB':
            image = image.convert('RGB')
        # Convert PIL image to OpenCV format
        cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    except Exception as e:
        print(f"Error converting image to OpenCV format: {e}")
        raise e  # Re-raise the exception

    try:
        # Detect features
        optic_nerve_pos = detect_optic_nerve(cv_image)
        macula_pos = detect_macula(cv_image)
        notch_present = detect_notch(cv_image)
    except Exception as e:
        print(f"Error during feature detection: {e}")
        raise e

    # Ensure features were detected
    if optic_nerve_pos is None or macula_pos is None:
        print("Could not detect optic nerve or macula.")
        return False  # Or handle as per your needs

    # Calculate optic nerve midline (y-coordinate)
    optic_nerve_midline_y = optic_nerve_pos['y']

    # Compare macula position to optic nerve midline
    macula_higher = macula_pos['y'] < optic_nerve_midline_y

    # Determine inversion based on criteria
    if (macula_higher) or (not notch_present):
        # Image is inverted
        return True
    else:
        # Image is not inverted
        return False

def correct_inversion(image):
    # Flip the image vertically
    corrected_image = image.transpose(Image.FLIP_TOP_BOTTOM)
    return corrected_image


    #<<functions for feature detection>>

    # Optic never detection
def detect_optic_nerve(image):
    if len(image.shape) == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    else:
        gray_image = image

    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(gray_image, (5, 5), 0)

    # Apply thresholding to segment bright areas
    _, thresh = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Assume the largest bright area is the optic nerve
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        # Calculate the center of the optic nerve
        M = cv2.moments(largest_contour)
        if M['m00'] != 0:
            cx = int(M['m10'] / M['m00'])  # x-coordinate
            cy = int(M['m01'] / M['m00'])  # y-coordinate
            return {'x': cx, 'y': cy}
    return None


#Macula detection

def detect_macula(image):
    # Convert to grayscale if necessary
    if len(image.shape) == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image

    # Image dimensions
    height, width = gray_image.shape

    # Define the central region (e.g., central 50% of the image)
    x_start = int(width * 0.2)
    x_end = int(width * 0.75)
    y_start = int(height * 0.2)
    y_end = int(height * 0.75)

    # Crop the central region
    central_region = gray_image[y_start:y_end, x_start:x_end]

    #Gaussian Blur 
    blurred = cv2.GaussianBlur( central_region, (9, 9), 0,cv2.BORDER_DEFAULT)

    #Thresholding
    res,thresh = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY_INV)

    # Filter

    # _, otsu_thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) Otsu's Thresholding could be used for better results

    #Contours
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        print("No dark regions detected in the central area.")
        return {'x': 0, 'y': 0}
    
    # Assume the largest dark contour corresponds to the macula
    largest_contour = max(contours, key=cv2.contourArea)

    # Calculate the centroid of the largest contour
    M = cv2.moments(largest_contour)
    if M['m00'] == 0:
        print("Zero division error while calculating centroid.")
        return {'x': 0, 'y': 0}

    cX = int(M['m10'] / M['m00']) + x_start
    cY = int(M['m01'] / M['m00']) + y_start
    

    
    # Create a resizable window
    cv2.namedWindow('Grayscale Image', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Grayscale Image', 800, 600)  # Set to desired dimensions
    cv2.namedWindow('Blurred Image', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Blurred Image', 800, 600)  
    cv2.namedWindow('Thresholded Central Region', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Thresholded Central Region', 800, 600)  
    cv2.namedWindow('Detected Macula', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Detected Macula', 800, 600)  
    cv2.namedWindow('Central Region', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Central Region', 800, 600)
    cv2.namedWindow('res', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Central Region', 800, 600) 
    cv2.circle(image, (cX, cY), 30, (0, 0, 255), -1)  # Red circle for macula
   

    cv2.imshow('Grayscale Image', gray_image)
    cv2.imshow('Blurred Image', blurred)
    cv2.imshow('Thresholded Central Region', thresh)
    cv2.imshow('res', res)
    cv2.imshow('Detected Macula', image)
    cv2.imshow('Central Region', central_region)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


    return {'x': cX, 'y': cY}

#Notch detection
def detect_notch(image):
    # Convert to grayscale if necessary
    if len(image.shape) == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    else:
        gray_image = image

    # Crop the edges of the image where the notch is expected
    height, width = gray_image.shape
    edge_width = int(width * 0.05)  # Adjust as needed

    # Left edge
    left_edge = gray_image[:, :edge_width]
    # Right edge
    right_edge = gray_image[:, -edge_width:]

    # Combine edges
    edges = [left_edge, right_edge]

    # Look for contours in the edge regions
    for edge in edges:
        # Apply thresholding
        _, thresh = cv2.threshold(edge, 35, 255, cv2.THRESH_BINARY)

        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Analyze contours
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 100:  # Adjust threshold based on expected notch size
                # Approximate contour to a polygon
                approx = cv2.approxPolyDP(cnt, 0.04 * cv2.arcLength(cnt, True), True)
                # Check for square, triangle, or circle
                if len(approx) == 3 or len(approx) == 4 or len(approx) > 8:
                    return True  # Notch detected
    return False  # No notch detected 













Main Loop


In [103]:
for index, row in labels_df.iterrows():
    image_name = row['image']
    print(f"Processing image {image_name}")
    image = load_image(image_name)

    if image is not None:
        try:
            inverted = is_inverted(image)
            labels_df.at[index, 'inverted'] = inverted
            print(f"Image {image_name} inverted: {inverted}")
        except Exception as e:
            print(f"Error processing image {image_name}: {e}")
            labels_df.at[index, 'inverted'] = None
    else:
        print(f"Skipping image {image_name} due to loading error.")
        labels_df.at[index, 'inverted'] = None


Processing image 10_left
No dark regions detected in the central area.
Image 10_left inverted: True
Processing image 10_right
No dark regions detected in the central area.
Could not detect optic nerve or macula.
Image 10_right inverted: False
Processing image 13_left
No dark regions detected in the central area.
Image 13_left inverted: True
Processing image 13_right
No dark regions detected in the central area.
Image 13_right inverted: True
Processing image 15_left
No dark regions detected in the central area.
Could not detect optic nerve or macula.
Image 15_left inverted: False
Processing image 15_right
No dark regions detected in the central area.
Image 15_right inverted: True
Processing image 16_left
No dark regions detected in the central area.
Image 16_left inverted: True
Processing image 16_right
No dark regions detected in the central area.
Image 16_right inverted: True
Processing image 17_left
No dark regions detected in the central area.
Image 17_left inverted: True
Processing

KeyboardInterrupt: 

Test bench

In [109]:
# Test on a sample image
sample_image = load_image(labels_df.iloc[8]['image'])
cv_image = cv2.cvtColor(np.array(sample_image), cv2.COLOR_RGB2BGR)

# Detect features
optic_nerve_pos = detect_optic_nerve(cv_image)
macula_pos = detect_macula(cv_image)
notch_present = detect_notch(cv_image)

print(f"Optic Nerve Position: {optic_nerve_pos}")
print(f"Macula Position: {macula_pos}")
print(f"Notch Present: {notch_present}")

# Display the image with detected points
if optic_nerve_pos and macula_pos:
    cv2.circle(cv_image, (optic_nerve_pos['x'], optic_nerve_pos['y']), 50, (0, 255, 0), -1)
    cv2.circle(cv_image, (macula_pos['x'], macula_pos['y']), 50, (0, 0, 255), -1)

     # Create a resizable window
    cv2.namedWindow('Detected Features', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Detected Features', 800, 600)  # Set to desired dimensions

    

    

    cv2.imshow('Detected Features', cv_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Optic Nerve Position: {'x': 1462, 'y': 1321}
Macula Position: {'x': 1846, 'y': 1230}
Notch Present: False
